In [5]:
cd utils/scrape_critics

/home/ubuntu/cv/aerial/DeepNetsEO/DeepNetsForEO/nlp/site/src/utils/scrape_critics


In [9]:
from scraper_functions import *

In [11]:
import os, errno, sys
from bs4 import BeautifulSoup
from requests import get
from textblob import TextBlob
import pandas as pd
from pyteaser import Summarize
from django.utils.encoding import smart_str, smart_unicode

In [12]:
IW_URL = 'http://www.indiewire.com/t/reviews/'

In [14]:
IWSoup = BeautifulSoup(get(IW_URL).text,'html.parser')

In [26]:
IWSoup.find_all('header',class_='entry-header')[0].a['href']

u'http://www.indiewire.com/2017/12/pitch-perfect-3-review-anna-kendrick-trish-sie-1201908606/'

In [37]:
name_str = IWSoup.find_all('header',class_='entry-header')[2].text

In [45]:
name_str[2:name_str.find('Review')-2]

u'Star Wars: The Last Jedi'

In [71]:
def IW():
    IW_URL = 'http://www.indiewire.com/t/reviews/'
    IWSoup = BeautifulSoup(get(IW_URL).text,'html.parser')
    iw_dict = {}
    for entry in IWSoup.find_all('header',class_='entry-header'):
        r_dict = {}
        name_str = entry.text
        name = name_str[2:name_str.find('Review')-2]
        
        r_dict['name'] = name
        link = entry.a['href']
        r_dict['link'] = link
        
        r_soup = BeautifulSoup(get(link).text,'html.parser')
        r_critic = r_soup.find_all('meta',attrs={'name':'author'})[0]['content']
        r_text = r_soup.find_all('meta',attrs={'name':'body'})[0]['content']
        r_sentiment = TextBlob(r_text).sentiment
        r_polarity = r_sentiment[0]
        r_subjectivity = r_sentiment[1]
        summary = Summarize(name,r_text)
        r_blurb = ' '.join(summary)
        r_thumbsup = 'Positive' if r_polarity >0 else 'Negative'
        
        r_dict['critic'] = r_critic
        r_dict['text'] = r_text
        r_dict['blurb'] = r_blurb
        r_dict['review_subjectivity'] = r_subjectivity
        r_dict['review_polarity'] = r_polarity
        r_dict['sentiment'] = r_sentiment
        r_dict['thumbsup'] = r_thumbsup
        
        
        iw_dict[name] = r_dict
        
    return iw_dict
    
    

In [72]:
iw = IW()

In [107]:
def reviewsDictToDatabase(mydict,name_of_publication):
    
    if Organization.objects.filter(name=name_of_publication).exists():
        print 'Organization %s already exists!' % (name_of_publication)
    else:
        newOrg = Organization(name=name_of_publication)
        newOrg.save()
        print 'added %s to the Organization Database' % (name_of_publication)
    
    myOrg = Organization.objects.get(name=name_of_publication)
    defaultDir = Director.objects.all()[0]
    
    for movie_review in mydict:
        r_name = mydict[movie_review]['name']
        r_critic = mydict[movie_review]['critic']
        r_link = mydict[movie_review]['link']
        r_text = mydict[movie_review]['text']
        r_blurb = mydict[movie_review]['blurb']
        r_sentiment = mydict[movie_review]['sentiment']
        r_thumbsup = mydict[movie_review]['thumbsup']
        r_subjectivity = mydict[movie_review]['review_subjectivity']
        r_polarity = mydict[movie_review]['review_polarity']
        if Movie.objects.filter(name=r_name).exists():
            print '%s movie already exists in the Database' % (r_name)
        else:
            newMovie = Movie(name=r_name,director=defaultDir)
            newMovie.save()
            print 'added %s to database' % (newMovie)
            
        if Critic.objects.filter(name=r_critic).exists():
            print '%s critic already exists in the database!' % (r_critic)
        else:
            newCritic = Critic(name=r_critic,organization=myOrg)
            print newCritic
            newCritic.save()
            print 'added %s from %s to critic database' % (r_critic, 'name_of_organization')
        
        print r_name
        
        mymovie = Movie.objects.get(name=r_name)
        mycritic = Critic.objects.get(name=r_critic,organization=myOrg)
        
        if Reviews.objects.filter(movie=Movie.objects.get(name=r_name),critic=Critic.objects.get(name=r_critic,organization=myOrg)).exists():
            print 'Review of %s from %s already exists!' % (name,r_author)
        else:
            print 'adding review'
            mymov = Movie.objects.get(name=r_name)
            mycritic = Critic.objects.get(name=r_critic,organization=myOrg)
            newReview = Reviews(movie=mymov,critic=mycritic,organization=myOrg,review_url=r_link,text = r_text, blurb = r_blurb, thumbsup = r_thumbsup, review_polarity = r_polarity, review_subjectivity = r_subjectivity)
            newReview.save()
    return newReview

In [108]:
a=reviewsDictToDatabase(iw,'IndieWire')

Organization IndieWire already exists!
Cuba and the Cameraman movie already exists in the Database
David Ehrlich critic already exists in the database!
Cuba and the Cameraman
adding review
The Breadwinner movie already exists in the Database
David Ehrlich critic already exists in the database!
The Breadwinner
adding review
Star Wars: The Last Jedi movie already exists in the Database
Eric Kohn critic already exists in the database!
Star Wars: The Last Jedi
adding review
Pitch Perfect 3 movie already exists in the Database
Kate Erbland critic already exists in the database!
Pitch Perfect 3
adding review
Voyeur movie already exists in the Database
David Ehrlich critic already exists in the database!
Voyeur
adding review
Jumanji: Welcome to the Jungle movie already exists in the Database
David Ehrlich critic already exists in the database!
Jumanji: Welcome to the Jungle
adding review
Justice League movie already exists in the Database
Eric Kohn critic already exists in the database!
Justi

In [111]:
a.movie

<Movie: Phantom Thread>

In [115]:
mypath = "/home/ubuntu/cv/aerial/DeepNetsEO/DeepNetsForEO/nlp/site/src/utils/scrape_critics"


In [116]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [118]:
onlyfiles.remove('__init__.py')

In [120]:
onlyfiles.remove('scraper_functions.py')
onlyfiles.remove('run_all_movie_scrapers.py')
onlyfiles.remove('scraper_functions.pyc')

In [121]:
onlyfiles

['NYT.py', 'variety.py', 'hollywood_reporter.py', 'indiewire.py']

In [124]:
cd utils/scrape_critics/

/home/ubuntu/cv/aerial/DeepNetsEO/DeepNetsForEO/nlp/site/src/utils/scrape_critics


In [126]:
from NYT import *

In [127]:
Dict = scrapeNYTMovie()

20


In [130]:
dictToDB(Dict)

A.O. Scott already exists! 
Quest already exists!
review of Quest from NYT already exists!
Jeannette Catsoulis already exists! 
The Rape of Recy Taylor already exists!
review of The Rape of Recy Taylor from NYT already exists!
Manohla Dargis already exists! 
Star Wars: The Last Jedi already exists!
review of Star Wars: The Last Jedi from NYT already exists!
A.O. Scott already exists! 
Wormwood already exists!
review of Wormwood from NYT already exists!
Glenn Kenny already exists! 
Beyond Skyline already exists!
review of Beyond Skyline from NYT already exists!
Ben Kenigsberg already exists! 
Killing for Love already exists!
review of Killing for Love from NYT already exists!
Ben Kenigsberg already exists! 
Bullet Head already exists!
review of Bullet Head from NYT already exists!
Glenn Kenny already exists! 
Youth already exists!
review of Youth from NYT already exists!
Andy Webster already exists! 
Sundowners already exists!
review of Sundowners from NYT already exists!
Glenn Kenny al